In [21]:
import requests
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv('TALLY_API_KEY')

url = f'https://api.tally.xyz/query'

headers = {
    "Content-Type": "application/json",
    "Api-Key": api_key
}

In [ ]:
query = """
query Proposals($input: ProposalsInput!) {
  proposals(input: $input) {
    nodes {
      ... on Proposal {
        onchainId
        creator {
          address
          type
        }
        metadata {
          title
          description
        }
        # proposer {
        #   address
        # }
        status
      }
    }
    pageInfo {
      count
      firstCursor
      lastCursor
    }
  }
}
"""

variables = {
    "input": {
        "filters": {
          "organizationId": 2206072049829414624
        }
    }
}

In [ ]:
all_proposals = []
after_cursor = None
has_more = True

while has_more and len(all_proposals) < 100:
    page_input = {
        "limit": 20
    }
    if after_cursor:
        page_input["afterCursor"] = after_cursor
        
    variables['input']['page'] = page_input
    
    response = requests.post(url, headers=headers, json={"query": query, "variables": variables})
    data = response.json()

    proposals = data["data"]["proposals"]["nodes"]
    all_proposals.extend(proposals)

    page_info = data["data"]["proposals"]["pageInfo"]
    after_cursor = page_info["lastCursor"]
    has_more = len(proposals) > 0 and after_cursor is not None

In [23]:
with open('../data/proposals/aave/aave_v2_tally_proposals.json', 'w', encoding='utf8') as f:
    json.dump(all_proposals, f, ensure_ascii=False, indent=4)

In [ ]:
query = """
query Organizations($input: OrganizationsInput) {
  organizations(input: $input) {
    nodes {
      ... on Organization {
        id
        name
        slug
        chainIds
      }
    }
    pageInfo {
      count
    }
  }
}
"""

variables = {
    "input": {
        "filters": {
            "chainId": "eip155:1"
        }
    }
}

response = requests.post(url, headers=headers, json={"query": query, "variables": variables})
data = response.json()

In [36]:
with open('../data/proposals/aave/aave_v2_on_chain_proposals_1_year.json', 'r', encoding="utf8") as file:
    v2_proposals = json.load(file)['data']['proposals']

In [ ]:
id_to_metadata = {proposal['onchainId']: proposal['metadata'] for proposal in all_proposals if 'onchainId' in proposal and 'metadata' in proposal}

for entry in v2_proposals:
    if entry['id'] in id_to_metadata:
        metadata = id_to_metadata[entry['id']]
        entry['title'] = metadata.get('title', '')
        entry['description'] = metadata.get('description', '')


In [40]:
with open('../data/proposals/aave/aave_v2_on_chain_proposals_1_year.json', 'w', encoding='utf8') as f:
    json.dump(v2_proposals, f, ensure_ascii=False, indent=4)